# Execution Constants and Helper Function

In [35]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

### Salvatore L Greco <salvatore@isi-technology.com>
### ISI Technology/RouteGenie Business Systems Analyst


ENV = {
    "HOST" : "https://your-subdomain.routegenie.com:8000/",
    "SECRET" : "",
    "CLIENTID" : ""
}

    
TESTID = "OtherPayerTest002"


PAYMENT_METHOD = {
    0 : 'Account',
    1 : 'Credit Card',
    2 : 'Cash',
    3 : 'Check',
    4 : 'Voucher',
    5 : 'Pass'
}

PAYMENT_STATUSES = {
    0 : 'Not paid',
    1 : 'Partial Paid',
    2 : 'Paid in Full',
}

"""Authentication and Create Order POST request test."""

import requests
import base64
from pprint import pprint

def get_access_token(client_secret, client_id):
    """Returns the base64 encoded access token credential.

    Parameters:
    client_secret (str): The RGAPI Client Secret.
    client_id (str): The RGAPI Client ID.
    """
    # Format the credential.
    credentials = f"{client_id}:{client_secret}"

    # Credentials is encoded from utf-8 to a base64 byte string.
    # The encoded credential is then converted from a byte string to a string.
    encoded_credentials = base64.b64encode(credentials.encode("utf-8")).decode("utf-8")

    return encoded_credentials


# Request Authentication Access Code

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

### Salvatore L Greco <salvatore@isi-technology.com>
### ISI Technology/RouteGenie Business Systems Analyst

HEADER = {
    "Authorization": f"Basic {get_access_token(ENV['SECRET'], ENV['CLIENTID'])}",
    "Content-Type": "application/x-www-form-urlencoded"
}

DATA = {'grant_type': 'client_credentials'}

token_request = requests.post(f"{ENV['HOST']}oauth2/token/", 
                              headers=HEADER, 
                              data=DATA)

access_token = token_request.json()['access_token']

# Print out the request status and request text
print(f"Authentication Response: [{token_request.status_code}]")
# print(token_request.text)


# Get Payers

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

### Salvatore L Greco <salvatore@isi-technology.com>
### ISI Technology/RouteGenie Business Systems Analyst


HEADER = {
  'Authorization': f'Bearer {access_token}',
  "Content-Type": "application/x-www-form-urlencoded"
}


DATA = {'grant_type': 'client_credentials'}

request = requests.get(f"{ENV['HOST']}open_api/api/v1/payer/", 
                              headers=HEADER)

# Print out the request status and request text
print(f"Payers: [{request.json()}]")
# print(equest.text)

# Get Trip Types

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

### Salvatore L Greco <salvatore@isi-technology.com>
### ISI Technology/RouteGenie Business Systems Analyst

HEADER = {
  'Authorization': f'Bearer {access_token}'
}

print(f"Requesting modes via {ENV['HOST']}open_api/api/v1/custom_trip_type/")

response = requests.get(f"{ENV['HOST']}open_api/api/v1/custom_trip_type/",
                        headers=HEADER)
raw_modes = response.json()['results']

print(f"Request Order Modes Response: [{response.status_code}]")

# Construct Modes dictionary

In [ ]:
ORDER_MODES = {pairs['id']:pairs['name'] for pairs in raw_modes}

print(ORDER_MODES)

# Request an Order

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

### Salvatore L Greco <salvatore@isi-technology.com>
### ISI Technology/RouteGenie Business Systems Analyst

DATA = {
    "ride_id": TESTID,
    "client_internal_id": 1,
    "custom_trip_type": 1,
    "payer_id": 30,
    "initial_time": "10/01/2025 16:30",
    "pick_up_address":"241 Main St Buffalo NY",
    "drop_off_address":"490 Delaware Ave Buffalo NY",
    "payment_method":1,
    "amount": 15
}
    
HEADER = {
    'Authorization': f'Bearer {access_token}'
}

print(f"Requesting {ORDER_MODES[DATA['custom_trip_type']]} ORDER {TESTID} for {DATA['client_internal_id']} at {DATA['initial_time']} from {DATA['pick_up_address']} to {DATA['drop_off_address']} paid by {PAYMENT_METHOD[DATA['payment_method']]}")
print(f"Requesting Order via {ENV['HOST']}open_api/api/v1/order/")
response = requests.post(f"{ENV['HOST']}open_api/api/v1/order/", 
                         headers=HEADER, 
                         data=DATA)

print(f"Create Order Response: [{response.status_code}]")
if (response.status_code==404):
    print("Order may already exist!")
print(f"[{response.text}]")

# Query the Order

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

### Salvatore L Greco <salvatore@isi-technology.com>
### ISI Technology/RouteGenie Business Systems Analyst

HEADER = {
    'Authorization': f'Bearer {access_token}'
}

PARAM = {
    "ride_id":TESTID
}

query_string = f"{ENV['HOST']}open_api/api/v1/order/{PARAM['ride_id']}/"

print(f"Querying ORDER {TESTID}")
queryorders = requests.get(query_string, 
                           headers = HEADER)

print(f"Query Order Response: [{queryorders.status_code}]")
DATA = queryorders.json()
#print(f'---\nSummary:\nORDER {TESTID} - {ORDER_MODES[DATA['custom_trip_type']]} for {DATA['client']['id']} at {DATA['initial_time']}\nfrom "{DATA['pick_up_address']}"\nto "{DATA['drop_off_address']}"\npaid by {PAYMENT_METHOD[DATA['fare_payment_info']['payment_method']]}')
#print(f"---\nStatus: Ride status - {DATA['status']} | Payment - {PAYMENT_STATUSES[DATA['fare_payment_info']['payment_status']]}\n---")
pprint(DATA, indent=4)

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

### Salvatore L Greco <salvatore@isi-technology.com>
### ISI Technology/RouteGenie Business Systems Analyst

HEADER = {
    'Authorization': f'Bearer {access_token}'
}

start_date = "10/01/2025"
end_date = "10/03/2025"
query_string = f'{ENV['HOST']}open_api/api/v1/order/?start_date={start_date}&end_date={end_date}?client_id=4231'

print(f"Querying ORDER {TESTID}")
queryorders = requests.get(query_string, 
                           headers = HEADER)

print(f"Query Order Response: [{queryorders.status_code}]")
DATA = queryorders.json()
#print(f'---\nSummary:\nORDER {TESTID} - {ORDER_MODES[DATA['custom_trip_type']]} for {DATA['client']['id']} at {DATA['initial_time']}\nfrom "{DATA['pick_up_address']}"\nto "{DATA['drop_off_address']}"\npaid by {PAYMENT_METHOD[DATA['fare_payment_info']['payment_method']]}')
#print(f"---\nStatus: Ride status - {DATA['status']} | Payment - {PAYMENT_STATUSES[DATA['fare_payment_info']['payment_status']]}\n---")
pprint(DATA, indent=4)